### 🏗️ Pipeline de Engenharia de Dados: Survey Visa/Embratur

**Objetivo:** Transformar dados brutos de pesquisa (formato *Wide*) em um Modelo Dimensional (Star Schema) otimizado para o Power BI.
**Entrada:** `codigo_dados.xlsx`
**Saída:** `d_respondente.csv`, `d_perguntas.csv`, `f_respostas.csv`

#### 1. Configuração e Extração (Extract)

**O que faz:** Carrega as bibliotecas, lê as abas do Excel e padroniza os nomes das colunas de metadados.
**Justificativa:** O Excel de origem possui nomes de colunas inconsistentes (ex: `descricao` em vez de `texto_pergunta`). Padronizar no início evita erros de *KeyError* nas etapas seguintes.

In [ ]:
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from typing import Tuple, Dict, Optional, Any
import re
from pathlib import Path
import os

FILENAME = 'codigo_dados.xlsx'

def configure_environment():
    warnings.filterwarnings('ignore')
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)

def load_excel_data(file_path: Path) -> Dict[str, pd.DataFrame]:
    """
    Carrega todas as abas necessárias do Excel de forma otimizada.
    Retorna um dicionário contendo os DataFrames.
    """
    if not file_path.exists():
        raise FileNotFoundError(f"Arquivo '{file_path.name}' não encontrado em: {file_path.parent}")

    try:
        with pd.ExcelFile(file_path) as xls:
            required_sheets = ['data', 'variable', 'datamap']
            missing_sheets = [s for s in required_sheets if s not in xls.sheet_names]
            
            if missing_sheets:
                raise ValueError(f"Abas ausentes: {missing_sheets}")

            return {
                'data': pd.read_excel(xls, sheet_name='data'),
                'variable': pd.read_excel(xls, sheet_name='variable'),
                'datamap': pd.read_excel(xls, sheet_name='datamap')
            }
    except Exception as e:
        raise RuntimeError(f"Falha crítica ao ler Excel: {e}")

def process_metadata(df_variable: pd.DataFrame) -> pd.DataFrame:
    """Padroniza a tabela de variáveis (metadados)."""
    df = df_variable.copy()
    if 'descricao' in df.columns:
        df = df.rename(columns={'descricao': 'texto_pergunta'})
    
    df.columns = ['variavel', 'posicao', 'texto_pergunta']
    return df

def process_datamap(df_datamap: pd.DataFrame) -> pd.DataFrame:
    """Higieniza a tabela de mapeamento (De/Para)."""
    df = df_datamap.copy()
    df.columns = ['variavel', 'codigo', 'label_resposta']
    
    for col in ['variavel', 'label_resposta']:
        df[col] = df[col].astype(str).str.strip()
        
    df['codigo'] = pd.to_numeric(df['codigo'], errors='coerce')
    df = df.dropna(subset=['codigo'])
    df['codigo'] = df['codigo'].astype(int)
    return df

def run_extraction_pipeline() -> Optional[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]]:
    configure_environment()
    file_path = Path.cwd() / FILENAME
    
    try:
        dfs = load_excel_data(file_path)
        df_data = dfs['data']
        df_variable = process_metadata(dfs['variable'])
        df_datamap = process_datamap(dfs['datamap'])
        return df_data, df_variable, df_datamap

    except Exception as e:
        print(f"Erro na execução do Pipeline: {e}")
        return None

if __name__ == "__main__":
    resultado = run_extraction_pipeline()
    
    if resultado:
        df_data, df_variable, df_datamap = resultado
    else:
        print("Falha na inicialização dos dados.")

🔄 1. Iniciando Pipeline: Carregando e Padronizando Dados...
   📂 Lendo arquivo: codigo_dados.xlsx...
   ✅ Arquivo carregado e processado com sucesso.
   ✅ Metadados e Datamap higienizados.


#### 2. Tratamento do Dicionário (Datamap)

Antes de traduzir os dados, precisamos garantir que o nosso "tradutor" (o Datamap) esteja limpo.

**Problema:** O Excel mistura números (`1`), textos (`"1"`) e floats (`1.0`), o que impede o Python de encontrar as chaves corretamente.

In [ ]:
TARGET_COLUMNS_PROFILE = (
    'Respondent_Serial', 'GENDER_NonBinary', 'resp_age', 'QUOTAGERANGE',
    'D1', 'D2', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36',
    'S5_1_PAIS', 'S5_1_ESTADO',
    'S3', 'S4', 'S6', 'S7', 'TIPO', 'TEMPORADA',
    'CurrentDay', 'CurrentMonth', 'CurrentYear',
    'GASTO_PESSOA', 'Q24_1_VALOR', 'Q24_1_MOEDA', 'D4_1_VALOR', 'D4_1_MOEDA'
)

DATE_COLS = ['CurrentYear', 'CurrentMonth', 'CurrentDay']
DEFAULT_YEAR = 2025

def process_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gera a coluna 'data_pesquisa' e 'Onda' a partir das colunas de dia/mês/ano.
    Remove as colunas originais de data após o processamento.
    """
    df_out = df.copy()
    
    if not set(DATE_COLS).issubset(df_out.columns):
        df_out['data_pesquisa'] = pd.NaT
        df_out['Onda'] = DEFAULT_YEAR
        return df_out

    try:
        df_out['data_pesquisa'] = pd.to_datetime(
            df_out['CurrentYear'].astype(str) + '-' + 
            df_out['CurrentMonth'].astype(str) + '-' + 
            df_out['CurrentDay'].astype(str),
            errors='coerce'
        )
        
        df_out['Onda'] = df_out['CurrentYear'].fillna(DEFAULT_YEAR).astype(int)
        df_out = df_out.drop(columns=DATE_COLS, errors='ignore')
        
    except Exception:
        df_out['Onda'] = DEFAULT_YEAR
        
    return df_out

def create_respondent_dimension(df_input: pd.DataFrame, target_cols: tuple) -> pd.DataFrame:
    """
    Orquestra a criação da tabela dimensão respondente:
    1. Filtra colunas existentes.
    2. Aplica tratamento de datas.
    """
    available_cols = list(set(target_cols).intersection(df_input.columns))
    
    if 'Respondent_Serial' not in available_cols:
        raise ValueError("Erro Crítico: A coluna 'Respondent_Serial' não foi encontrada!")

    d_respondente = df_input[available_cols].copy()
    return process_date_columns(d_respondente)

if 'df_data' in locals():
    d_respondente = create_respondent_dimension(df_data, TARGET_COLUMNS_PROFILE)
else:
    print("Erro: 'df_data' não encontrado.")

🔄 2. Criando Dimensão Respondente (d_respondente)...
   ✅ Tabela criada com sucesso: 8131 linhas, 27 colunas.


#### 3. Construção da Dimensão Respondente (`d_respondente`)

Esta tabela contém o **perfil único** de cada turista.

**Transformações:**

* **Data:** Unificação de colunas separadas (Dia/Mês/Ano).
* **Tradução Robusta:** Conversão de códigos (`1`, `2`) para texto (`Sim`, `Não`) usando lógica híbrida (Excel + Dicionários Manuais para corrigir falhas na origem).
* **Renomeação:** Padronização para *snake_case* (ex: `id_respondente`).

In [ ]:

MAP_EMPLOYMENT = {
    1: 'Empregado em tempo integral', 2: 'Empregado em tempo parcial', 3: 'Autônomo',
    4: 'Desempregado (procurando)', 5: 'Desempregado (não procurando)/Incapacitado', 
    6: 'Cuidador/Dono(a) de casa', 7: 'Aposentado(a)', 8: 'Estudante', 9: 'Trabalho Remoto'
}

MAP_EDUCATION = {
    1: 'Sem estudos', 2: 'Fundamental incompleto', 3: 'Fundamental completo',
    4: 'Médio incompleto', 5: 'Médio completo', 6: 'Superior incompleto',
    7: 'Superior completo', 8: 'Pós-Graduação', 9: 'Mestrado', 10: 'Doutorado'
}

MAP_COMPANIONS = {
    1: 'Apenas eu', 2: 'Companheiro(a)', 3: 'Filho(s)',
    4: 'Pais', 5: 'Parentes', 6: 'Amigos'
}

MAP_CURRENCY_NAMES = {
    1: 'ARS', 2: 'USD', 3: 'CLP', 4: 'PYG', 5: 'UYU', 6: 'EUR', 
    7: 'GBP', 8: 'CAD', 9: 'COP', 10: 'PEN', 11: 'MXN', 12: 'BRL'
}

EXCHANGE_RATES = {
    'ARS': 0.00069, 'USD': 1.0, 'CLP': 0.0011, 'PYG': 0.00015,
    'UYU': 0.0256, 'EUR': 1.18, 'GBP': 1.35, 'CAD': 0.73,
    'COP': 0.00026, 'PEN': 0.29, 'MXN': 0.056, 'BRL': 0.18
}

RENAME_SCHEMA = {
    'Respondent_Serial': 'id_respondente', 'GENDER_NonBinary': 'genero', 
    'resp_age': 'idade', 'QUOTAGERANGE': 'faixa_etaria', 
    'S5_1_PAIS': 'pais', 'S5_1_ESTADO': 'estado_residencia', 
    'S3': 'reside_brasil', 'S4': 'viagem_int_12m', 
    'S6': 'meio_transporte', 'S7': 'portao_entrada',
    'TIPO': 'tipo_turista', 'TEMPORADA': 'temporada', 
    'GASTO_PESSOA': 'gasto_total_calculado',
    'D1': 'status_emprego', 'D2': 'escolaridade',
    'Q24_1_VALOR': 'gasto_viagem_original', 'Q24_1_MOEDA': 'moeda_viagem',
    'D4_1_VALOR': 'renda_familiar_original', 'D4_1_MOEDA': 'moeda_renda'
}

def get_mapping_for_column(col_name: str, df_map: pd.DataFrame) -> Dict[int, str]:
    subset = df_map[df_map['variavel'] == col_name]
    return {} if subset.empty else dict(zip(subset['codigo'], subset['label_resposta']))

def safe_translate(value: Any, mapping: Dict[Any, str]) -> str:
    if pd.isna(value) or str(value).strip() == '':
        return value
        
    val_str = str(value)
    
    if ';' not in val_str:
        try:
            code = int(float(val_str.split('.')[0]))
            return str(mapping.get(code, val_str))
        except ValueError:
            return val_str

    translated_parts = []
    for p in val_str.split(';'):
        try:
            code = int(float(p.strip()))
            translated_parts.append(str(mapping.get(code, p)))
        except ValueError:
            translated_parts.append(p)
            
    return "; ".join(translated_parts)

def calculate_usd_vectorized(df: pd.DataFrame, val_col: str, currency_col: str, result_col: str) -> pd.DataFrame:
    if val_col not in df.columns or currency_col not in df.columns:
        return df

    currency_codes = df[currency_col].map(MAP_CURRENCY_NAMES).fillna(df[currency_col])
    currency_codes = currency_codes.astype(str).str.upper()
    
    rates = currency_codes.map(EXCHANGE_RATES)
    mask_nan = rates.isna()
    
    if mask_nan.any():
        def infer_rate(text):
            if 'REAL' in text: return 0.18
            if 'EURO' in text: return 1.18
            if 'PESO ARG' in text: return 0.00069
            return 1.0 
        rates[mask_nan] = currency_codes[mask_nan].apply(infer_rate)

    df[result_col] = (df[val_col] * rates).round(2)
    return df

def run_transformation_pipeline(df: pd.DataFrame, df_map: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()

    cols_to_translate = [
        'GENDER_NonBinary', 'TEMPORADA', 'TIPO', 'S5_1_PAIS', 'S3', 'S4', 
        'S6', 'S7', 'QUOTAGERANGE', 'D1', 'D2', 
        'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 
        'Q24_1_MOEDA', 'D4_1_MOEDA'
    ]

    for col in cols_to_translate:
        if col not in df_out.columns:
            continue
            
        mapping = {}
        if col == 'D1': mapping = MAP_EMPLOYMENT
        elif col == 'D2': mapping = MAP_EDUCATION
        elif col.startswith('D3'): mapping = MAP_COMPANIONS
        elif 'MOEDA' in col: mapping = MAP_CURRENCY_NAMES
        else:
            mapping = get_mapping_for_column(col, df_map)
            
        if mapping:
            df_out[col] = df_out[col].apply(lambda x: safe_translate(x, mapping))

    df_out = calculate_usd_vectorized(df_out, 'Q24_1_VALOR', 'Q24_1_MOEDA', 'gasto_viagem_usd')
    df_out = calculate_usd_vectorized(df_out, 'D4_1_VALOR', 'D4_1_MOEDA', 'renda_familiar_usd')

    df_out = df_out.rename(columns=RENAME_SCHEMA)
    
    if 'estado_residencia' in df_out.columns:
        df_out['estado_residencia'] = (
            df_out['estado_residencia']
            .astype(str).str.title().str.strip()
            .replace({'Nan': 'Não Informado', 'Null': 'Não Informado'})
        )

    return df_out

if 'd_respondente' in locals() and 'df_datamap' in locals():
    d_respondente = run_transformation_pipeline(d_respondente, df_datamap)
else:
    print("Erro: Dependências não encontradas.")

🔄 3. Executando Transformação e Normalização...
   💰 Convertendo valores para USD...
   🏷️ Renomeando colunas...
   ✅ Transformação concluída.


#### 4. Construção da Fato Respostas (`f_respostas`)

Esta é a tabela longa (Unpivoted) que contém todas as respostas. 

**Otimizações:**

* **Melt:** Transforma colunas em linhas.
* **Limpeza de Nulos:** Remove linhas vazias para reduzir drasticamente o tamanho do arquivo.
* **Fallback de Texto:** Se não houver tradução (ex: "Outro país: Chile"), mantém o texto original digitado pelo usuário.

In [ ]:
IGNORE_COLS = [
    'Respondent_Serial', 'CurrentDay', 'CurrentMonth', 'CurrentYear', 
    'GENDER_NonBinary', 'resp_age', 'QUOTAGERANGE', 'S5_1_PAIS', 'S5_1_ESTADO', 'TIPO',
    'Q24_1_VALOR', 'Q24_1_MOEDA', 'D4_1_VALOR', 'D4_1_MOEDA' 
]

FIX_ACCOMMODATION = {
    1: 'Hotel de 3 estrelas ou menos', 2: 'Hotel de 4 ou 5 estrelas', 3: 'Pousada',
    4: 'Resort', 5: 'Casa de amigos e parentes', 6: 'Imóvel próprio',
    7: 'Imóvel alugado por temporada', 8: 'Airbnb e similares', 9: 'Albergue/hostel',
    10: 'Camping', 11: 'Cruzeiro / Navio', 98: 'Outros'
}

FIX_GASTRONOMY = {
    1: 'Restaurante de alta gastronomia', 2: 'Restaurante por quilo/buffet', 3: 'Fast Food',
    4: 'Cafeterias', 5: 'Étnicos', 6: 'Bistrô',
    7: 'Food truck', 8: 'Restaurantes à la carte', 98: 'Outros'
}

FIX_ACTIVITIES = {
    1: 'Tratamentos médicos/estéticos', 2: 'Terapias de bem-estar',
    4: 'Turismo místico/esotérico', 5: 'Atividades náuticas',
    6: 'Cruzeiros', 7: 'Visitas culturais',
    8: 'Comunidades tradicionais', 9: 'Eventos culturais',
    10: 'Gastronomia', 11: 'Esportes (geral)',
    12: 'Eventos esportivos', 13: 'Mergulho',
    14: 'Aventura (trilhas, rafting)', 15: 'Observação fauna/flora',
    16: 'Turismo rural', 17: 'Parques temáticos',
    18: 'Ecoturismo', 19: 'Compras',
    20: 'Visitar amigos/parentes', 21: 'Vida Noturna',
    22: 'Sol e praia', 23: 'Carnaval de rua',
    24: 'Carnaval sambódromo', 98: 'Outros', 99: 'Não realizou atividades'
}

MAP_UF_NAME = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapa', 'AM': 'Amazonas', 'BA': 'Bahia', 'CE': 'Ceara',
    'DF': 'Distrito Federal', 'ES': 'Espirito Santo', 'GO': 'Goias', 'MA': 'Maranhao', 'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Para', 'PB': 'Paraiba', 'PR': 'Parana',
    'PE': 'Pernambuco', 'PI': 'Piaui', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul', 'RO': 'Rondonia', 'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'Sao Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}

def unpivot_data(df: pd.DataFrame, ignore_list: list) -> pd.DataFrame:
    """Transforma colunas em linhas (Melt) e remove nulos."""
    cols_to_melt = [c for c in df.columns if c not in ignore_list]
    
    df_melt = df.melt(
        id_vars=['Respondent_Serial'], 
        value_vars=cols_to_melt, 
        var_name='cod_pergunta', 
        value_name='cod_resposta'
    )
    
    df_melt = df_melt.dropna(subset=['cod_resposta'])
    df_melt = df_melt[df_melt['cod_resposta'].astype(str).str.strip() != '']
    return df_melt

def apply_translations(df_fact: pd.DataFrame, df_map: pd.DataFrame) -> pd.DataFrame:
    """Realiza o Left Join com o Datamap para trazer os textos das respostas."""
    merged = df_fact.merge(
        df_map, 
        left_on=['cod_pergunta', 'cod_resposta'], 
        right_on=['variavel', 'codigo'], 
        how='left'
    )
    merged['resposta_texto'] = merged['label_resposta'].fillna(merged['cod_resposta'])
    return merged

def patch_specific_questions(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica correções manuais (Dicionários) baseadas em Regex de perguntas."""
    df_out = df.copy()

    def _translate_item(val, mapping):
        val_str = str(val).strip()
        try:
            return mapping.get(int(float(val_str)), val_str)
        except:
            if ';' in val_str:
                parts = [str(mapping.get(int(float(p)), p)) if p.replace('.','',1).isdigit() else p for p in val_str.split(';')]
                return "; ".join(parts)
            return val_str

    patches = [
        (r'^Q15|^Q16', FIX_ACCOMMODATION),
        (r'^Q18', FIX_GASTRONOMY),
        (r'^Q23', FIX_ACTIVITIES)
    ]

    for pattern, mapping in patches:
        mask = df_out['cod_pergunta'].astype(str).str.contains(pattern, regex=True, na=False)
        if mask.any():
            df_out.loc[mask, 'resposta_texto'] = df_out.loc[mask, 'cod_resposta'].apply(lambda x: _translate_item(x, mapping))

    return df_out

def clean_and_enrich(df: pd.DataFrame) -> pd.DataFrame:
    """Limpeza final de textos (Outros) e criação de colunas geográficas."""
    df_out = df.copy()
    
    df_out['resposta_texto'] = df_out['resposta_texto'].astype(str).str.replace(r'^Outros.*', 'Outros', regex=True)
    df_out['UF_Mapa'] = df_out['resposta_texto'].str.extract(r'\((.*?)\)')
    df_out['Nome_Estado_Mapa'] = df_out['UF_Mapa'].map(MAP_UF_NAME)
    
    df_out = df_out.rename(columns={'Respondent_Serial': 'id_respondente'})
    cols_final = ['id_respondente', 'cod_pergunta', 'cod_resposta', 'resposta_texto', 'Nome_Estado_Mapa']
    return df_out[cols_final]

def create_fact_responses(df_raw_input: pd.DataFrame, df_map_input: pd.DataFrame) -> pd.DataFrame:
    f_respostas = unpivot_data(df_raw_input, IGNORE_COLS)
    f_respostas = apply_translations(f_respostas, df_map_input)
    f_respostas = patch_specific_questions(f_respostas)
    f_respostas = clean_and_enrich(f_respostas)
    return f_respostas

if 'df_data' in locals() and 'df_datamap' in locals():
    f_respostas = create_fact_responses(df_data, df_datamap)
else:
    print("Erro: Dependências não encontradas.")

🔄 4. Criando Fato Respostas (f_respostas)...
   🔥 Unpivoting (Melt) dos dados...
   📖 Traduzindo códigos via Datamap...
   🔧 Aplicando patches (Acomodação, Gastronomia, Atividades)...
   🌍 Higienizando textos e gerando dados geográficos...
   ✅ f_respostas criada com 1443985 linhas.


#### 5. Construção da Dimensão Perguntas (`d_perguntas`)

Tabela de metadados para criar menus e filtros no Dashboard.

**Transformações:**

* **Categorização:** Criação da coluna `categoria` baseada nos prefixos (Q1, Q24, S...) para permitir navegação por menu.
* **Limpeza:** Remoção de tags técnicas (`[HIDDEN]`) e caracteres indesejados.

In [ ]:
import re

CATEGORY_RULES = [
    ('1. Perfil do Turista',        ['D', 'GENDER', 'RESP_AGE', 'QUOTA', 'S5_', 'Q1', 'Q2']),
    ('2. Planejamento e Marketing', ['Q3', 'Q4', 'Q5', 'Q6', 'Q23A']),
    ('3. Características da Viagem',['S', 'Q7', 'Q8', 'Q9', 'Q10', 'Q14', 'Q19', 'Q20', 'Q21', 'Q22']),
    ('4. Destinos Visitados',       ['Q11', 'Q12', 'Q13']),
    ('5. Hospedagem e Transporte',  ['Q15', 'Q16', 'Q17']),
    ('6. Atividades e Gastronomia', ['Q18', 'Q23']),
    ('7. Gastos e Pagamentos',      ['Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'GASTO']),
    ('8. Avaliação e Imagem',       ['Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39'])
]

TECHNICAL_JUNK = [
    'TIPO', 'RESPONDENT_SERIAL', 'CURRENTDAY', 'CURRENTMONTH', 'CURRENTYEAR'
]

def get_category(code: str) -> str:
    """Define a categoria baseada no código da pergunta."""
    code_upper = str(code).upper().strip()
    
    for category, prefixes in CATEGORY_RULES:
        if any(code_upper.startswith(p) for p in prefixes):
            if code_upper.startswith('D') and not (len(code_upper) > 1 and code_upper[1].isdigit()):
                continue 
            return category
            
    return '9. Outros / Técnico'

def clean_question_text(text_series: pd.Series) -> pd.Series:
    """Limpeza vetorizada de textos das perguntas (Regex)."""
    clean = text_series.astype(str).str.strip()
    clean = clean.str.replace(r'\s*:\s*$', '', regex=True)
    clean = clean.str.replace(r'\[.*?\]', '', regex=True)
    clean = clean.str.replace(r'\s+', ' ', regex=True).str.strip()
    return clean

def create_dim_questions(df_vars: pd.DataFrame, df_fact: pd.DataFrame) -> pd.DataFrame:
    df_dim = df_vars[['variavel', 'texto_pergunta']].copy()
    df_dim.columns = ['cod_pergunta', 'texto_pergunta']
    
    df_dim['categoria'] = df_dim['cod_pergunta'].apply(get_category)
    
    valid_questions = set(df_fact['cod_pergunta'].unique())
    
    is_junk = df_dim['cod_pergunta'].str.upper().isin(TECHNICAL_JUNK)
    is_coded = df_dim['cod_pergunta'].str.contains('_Coded_|_SEM_OUTLIERS', na=False)
    is_orphan = ~df_dim['cod_pergunta'].isin(valid_questions)
    
    df_dim = df_dim[~(is_junk | is_coded | is_orphan)].copy()
    
    df_dim['texto_pergunta'] = clean_question_text(df_dim['texto_pergunta'])
    
    return df_dim.sort_values(by=['categoria', 'cod_pergunta'])

if 'df_variable' in locals() and 'f_respostas' in locals():
    d_perguntas = create_dim_questions(df_variable, f_respostas)
else:
    print("Erro: Dependências não encontradas.")

🔄 5. Criando Dimensão Perguntas (d_perguntas)...
   🗂️ Categorizando perguntas para menu de navegação...
   🧹 Removendo perguntas técnicas e sem dados...
   ✅ d_perguntas criada com 409 perguntas categorizadas.


#### 6. Exportação (Load)

Salva os Dataframes em CSV prontos para importação no Power BI. Usamos `;` como separador para evitar conflitos com vírgulas nos textos das perguntas.

In [ ]:

EXPORT_CONFIG = {
    'sep': ';',
    'encoding': 'utf-8-sig',
    'index': False,
    'date_format': '%Y-%m-%d'
}

BASE_PATH = Path.cwd()

def clean_text_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Higieniza colunas de texto removendo quebras de linha e espaços extras.
    Trata valores nulos antes da conversão para string para evitar 'nan' literais.
    """
    df_out = df.copy()
    text_cols = df_out.select_dtypes(include=['object', 'string']).columns
    
    if len(text_cols) > 0:
        # Preenche nulos com vazio antes de converter para string
        df_out[text_cols] = df_out[text_cols].fillna('')
        
        for col in text_cols:
            df_out[col] = (
                df_out[col].astype(str)
                .str.replace(r'[\r\n]+', ' ', regex=True)
                .str.replace(r'\s+', ' ', regex=True)
                .str.strip()
            )
            
    return df_out

def export_csv(df: pd.DataFrame, filename: str):
    """Aplica limpeza final e salva o arquivo com as configurações padrão."""
    try:
        df_clean = clean_text_columns(df)
        file_path = BASE_PATH / filename
        df_clean.to_csv(file_path, **EXPORT_CONFIG)
    except PermissionError:
        print(f"Erro de Permissão: O arquivo '{filename}' está aberto no Excel/Power BI.")
    except Exception as e:
        print(f"Erro crítico ao salvar '{filename}': {e}")

def run_load_pipeline():
    artifacts = {
        'd_perguntas.csv': d_perguntas if 'd_perguntas' in locals() else None,
        'd_respondente.csv': d_respondente if 'd_respondente' in locals() else None,
        'f_respostas.csv': f_respostas if 'f_respostas' in locals() else None
    }
    
    for filename, df_obj in artifacts.items():
        if df_obj is not None:
            export_csv(df_obj, filename)
        else:
            print(f"Aviso: DataFrame para '{filename}' não encontrado na memória.")

if __name__ == "__main__":
    run_load_pipeline()

🔄 6. Iniciando Higienização Final e Exportação...
   🧹 Limpando tabela: d_respondente...
   🧹 Limpando tabela: d_perguntas...
   🧹 Limpando tabela: f_respostas...
   💾 Salvando arquivos CSV...
--------------------------------------------------
🚀 SUCESSO TOTAL! Pipeline concluído.
   Arquivos gerados na pasta:
   1. d_perguntas.csv
   2. d_respondente.csv
   3. f_respostas.csv
--------------------------------------------------
